# Table of content
[5.1. Why object recognition is difficult](#object_reg)  
[5.2. Ways to achieve viewpoint invariance](#viewpoint_invar)  
[5.3. Convolutional neural networks for hand written digit recognition](#hand_written_digit_reg)  
[5.4. Convolutional neural networks for object recognition](#conv_net_obj_reg)  

## 5.1. Why object recognition is difficult
<a id="object_reg"> </a>
- Things that make object recognition hard to recognize objects:  
    - Segmentation: real scenes are mixed with other objects:  
        - hard to tell which pieces go together as parts of the same object.
        - parts of an object can be hidden behind other object (suffers from the occlusion due to other objects).
    - Lighting: pixel intensity is as dependent on object as it is on lighting $\rightarrow$ variations in perspective lighting
    - Scale (Deformation): objects can deform in a variety of non-affine ways (i.e. a hand-written $2$ can have a large loop or just a cusp). Same object can look very different (for example, written numeral $2$ or $4$).
    - Affordances: object classes are often defined by how they are used (i.e. chairs are things designed for sitting on so they have a wide variety of physical shapes. You sit in a chair, but modern vs classic chairs can be widly different, then you have to have knowledge that the thing is to be sat on).  
    $\rightarrow$ many objects are defined more by what it is used for than what it looks like
    - Viewpoint/Transformation: changes in viewpoint cause changes in images that standard learning methods cannot cope with. Information hops (n~ bước nhảy thông tin) between input dimensions (i.e. pixels)
    ![infor_hops](images/infor_hops.png)
    i.e. A medical database in which the age of the patient is sometimes labeled incorrectly as the patient's weight - the example gives age and weight randomly changing locations - this is called "dimension hopping" which needs to be eliminated before applying ML. Viewpoint changes cause "dimension hopping".

## 5.2. Ways to achieve viewpoint invariance
<a id="viewpoint_invar"> </a>
- A few common approaches:
    - Use redundant invariant features
    - Box objects and normalize pixels: put a box around the object and use normalized pixels
    - Replicated features with pooling (convolutional neural nets - $\color{red}{Lecture\ 5.3}$
    - Hierarchy of parts that have explicit poses relative to camera ($\color{red}{Lecture\ 5.e}$)
- Details:
    - Invariant feature approach: 
        - Extract a large, redundant set of features that are invariant under transformations. 
        - The underlying assumption is based on the observation that humans can effortlessly detect objects in different poses and lighting conditions and, so, there must exist properties or features which are invariant over these variabilities. 
        - With enough invariant features, there is only one way to put them together into an image (relationships between features are automatically captured by other features due to multiple overlaps). 
        - But for recognition, need to avoid features that are parts of objects.
    - _Judicious normalization approach_ (boxing/normalizing objects):
        - Put a box around the object and use it as a coordinate frame for a set of normalized pixels.
        - Solves "dimension hopping" if the box is always done correctly, the same part of an object always occurs on the same normalized pixels.
        - Can provide invariance to many degrees of freedom: $\color{red}{translation,\ rotation,\ scale,\ shear\ (dich\ chuyen),\ stretch\ ...}$
        - Boxing, however is difficult, due to segmentation errors, occlusion, unusual orientations.
        - Need to know what the shape is in order to box it right, which is the problem looking to solve already.
    - _Brute Force normalization approach_ (boxing):
        - When training the recognizer, use very clean data (use well-segmented, upright images) for training, to fit the correct box, so boxing can be done accurately and cleanly.
        - At test time, try all possible boxes in a range of positions and scales, try to throw noisier less clean data.
        - Is widely used for detecting upright things like faces and house numbers in unsegmented images.
        - Important that the network can tolerate some sloppiness in the boxing so more coarse/less accurate boxing can be dont at test time. (it is much more efficient if the recognizer can cope with some variation in position and scale so that we can use a coarse grid when trying all possible boxes).

## 5.3. Convolutional neural networks for hand written digit recognition
<a id="hand_written_digit_reg"> </a>

### 5.3.1. The replicated feature approach (currently the dominant approach for NNs)
- The convolutional neural network is based on the idea of the replicated features.  
    - use many different copies of the __same feature__ detector with different positions.  
        - could be replicated across the scale and orientation (tricky and expensive) $\rightarrow$ no good.  
        - replication helps to reduce the number of free parameters that are have to learned.
    - when you use several __different feature types__, each of them will has its own map of replicated detectors (each has its own convolution function).  
        - each map will have replicas of the same feature.
        - feature is constrained to be identical in different places.
        - different map will learn to detect different features.

### 5.3.2. The backpropagation with weight constraints
- The replicated features fit with the backpropagation (modify the backpropagation algorithm to incorporate linear constraints between the weights).
- Compute the gradients as usual, then modify the gradients so that satisfy the linear constraints, so they will also satisfy the linear constraints before and after the weight update.  
    Ex: To constrain: $w_1=w_2$, we need $\Delta w_1=\Delta w_2$ and start off with $w_1=w_2$.  
    The way we do is that we compute the gradient of the error w.r.t. $w_1, w_2$: $$\frac{\partial E}{\partial w_1}, \frac{\partial E}{\partial w_2}$$.
    Then we use the sum or the average of both gradients for weight update, $\frac{\partial E}{\partial w_1} + \frac{\partial E}{\partial w_2}$ for $w_1, w_2$.
    By using the weight constraints like this, we can force back propagation to learn replicated feature detectors.

### 5.3.3 What does replicating the feature detectors achieve?
- Equivariant activities: what replicated features achieved is the equivariance, not invariance in the image translation.
    ![translated_image](images/translated_image.png)
    when the image is translated, the black dots are also translated, so the image is translated and the representation also changed as much as. This is the equivariance, not invariance.
- Invariant knowledge: There is something that is invariant, which is the knowledge. Therefore, if you learn replicate feature detectors, and you know how to detect a feature in a place, you'll know how to detect that same feature in another place.  
    If a feature is useful in some locations during __*training*__, detectors for that feature will be available in all locations during __*testing*__.  
    **Summary**: Replicating the feature achieves equivariance in the activities, but invariance in the weights.

### 5.3.4. Pooling the outputs of replicated feature detectors
- If you want to achieve some invariance in the activities, how we do?  
- It is that you pool the outputs of replicated feature detectors.
- Because of this, you can get a small amount of translational invariance at each level of a deep net, by averaging **four** neighboring replicated detectors to give a single output to the next layer.  
    - Due to this averaging, it reduces the number of inputs for the next layer of feature extraction, thus allowing us to have more different feature maps, allowing us to learn more different kinds of features in the next layer.  
    - The pooling can be done by averaging or taking the maximum of **four** neighboring replicated detectors.  
- **Problem**: After several levels of pooling, we have lost precise information about _where_ (the precise positions of) things are. This makes it is impossible to use the precise spatial relationships between high-level parts for recognition.  
    Example: For face detecting, if you want to recognize whose face it is, you need to use the precise spatial relationships between some components of the face (e.g. between the eyes, between the nose and the mouth), which is lost by these convolutional neural nets.  
    It means that allowing us to recognize if the image is a face, but "if you want to recognize whose face it is", you need the precise spatial relationships between high-level parts, which have been lost in CNNs.

### 5.3.5. The convolutional neural networks for recognition of hundred digits
(based on the work of Yann LeCun and his collaborators)
- This is a really good recognizer for handwritten digits based on backpropagation in a feedforward net with:  
    - many hidden layers.  
    - in each hidden layer, there are many maps of replicated units.
    - pooling between layers, in specific, pooling the outputs of adjacent replicated units before sending them into the next layer.
    - it is a wide net which can cope with several characters at the same time and that works even if characters overlapped. $\rightarrow$ do not need to segment out individual characters before feeding them into this net.
    - use a clever way is to train a complete system in which you put in pixels at one end and you get out whole numbers (zip codes) at the other end, not just a recognizer of individual characters. This training is called __maximum margin__.
- This net coped with variations in size, orientation, position, overlap of digits and all kinds of background noise that would kill most other methods.
- The architecture of LeNet-5 ([homepage for demo](http://yann.lecun.com/exdb/lenet/index.html))
![arch_lenet5](images/arch_lenet5.png)
Apply convolution on 2D images and use backpropagation.
Structure: 2 convolutional layers (with pooling) denoted as C and S + 3 fully connected layers
    - convolution kernel size (C): 5x5
    - pooling (sub-sampling) (S): 2x2
    - there is an input, which is pixels (32x32 images).
        - input size: 32x32x1
    - then, the whole sequence of feature maps:
        - in C1 feature maps, there are 6 different maps, each of them is 28x28 pixels (6 @ 28x28 pixels each). Each pixel in one of these maps (28x28) pixels is computed by applying the (5x5) pixels convolution function to original image and their weights are constrained together, i.e. all (5x5) pools are the same, so there are only 9 parameters per map.
            - C1 size: 28x28x6
![arch_lenet5_convo](images/arch_lenet5_convo.png)
        - followed by is S2 feature maps = sub sampling which is called as pooling where pool together the outputs of a bunch of neighboring replicated features in C1. With the (2x2) convolution function, this layer reduces each (28x28) down to (14x14).
            - S2 size: 14x14x6
![arch_lenet5_pooling](images/arch_lenet5_pooling.png)
        - as you go up this hierarchy, you get features being much more complicated, but being more invariant to position. It is the **errors** that LeNet5 made.
            - C3 size: 10x10x16
![arch_lenet5_C3](images/arch_lenet5_C3.png)
Each unit in each feature map of C3 is connected to several (5x5) neighborhoods at identical locations in a subset of S2's feature maps.
![arch_lenet5_C3_S2](images/arch_lenet5_C3_S2.png)
As shown in the above table, the first six C3 feature maps take inputs from every contiguous subsets of three feature maps in S2. The next six take input from every contiguous subsets of four. The next three take input from some discontinuous subsets of four. Finally the last one takes input from all S2 feature maps.
            - S4 size: 5x5x16
![arch_lenet5_S4](images/arch_lenet5_S4.png)
            - C5 size: 120, is a convolutional layer with 120 feature maps. Each unit is connected to a (5x5) neighborhood on all 16 of S4's feature maps. Because the size of S4 is also (5x5), the size of C5's feature maps is (1x1).
![arch_lenet5_C5](images/arch_lenet5_C5.png)
            - F6 size: 84 units (fully connected to C5)
    - finally, the output layer has 10 units (one for each digit, fully connected with F6)
![arch_lenet5_output](images/arch_lenet5_output.png)
- LeNet5 run on 10,000 cases and made 82 errors, which means it corrected 99,18%.

### 5.3.6. Priors and Prejudice
- How to provide prior knowledge in machine learning, particularly to neural networks?
    - can put our prior knowledge about the task into the network by designing appropriate:
        - local connectivity.
        - weight constraints.
        - neuron activation functions which are paricularly appropriate for the task.
    - those above are _less instructive than hand-designing the features_
        - but it still prejudices (dinh huong) the network towards the particular way of solving the problem that we had in mind.
            - Example: we have an idea of how to do object recognition by gradually making bigger and bigger features and by replicating these features across space. We forced the nets to train this way (LeNet5).
    - alternatively, can use our prior knowledge to create a whole lot more training data.
        - Example: model the simulation of the steel mill (in Hofman & Tresp's paper) written by Fortran language.
        - but it may make learning take much longer.
            - Example: in terms of the speed of learning, it is more efficient to put prior knowledge by connectivity and weight constraints (like LeNet5). But if the computers are faster, generating more synthetic examples (more training data) begins to look better and better.
        - allows optimization to discover clever ways of using the multi-layer nets that we did not think of.

### 5.3.7. The brute force approach
- If you want to have a good solution, by using the idea of synthetic data.
- There is a brute force approach for handwritten digit recognition.
    - LeNet5 use the knowledge of invariances to design:
        - the local connectivity
        - the weight-sharing
        - the pooling
    - This achieved about 80 errors
        - can be reduced to about 40 errors by using many different transformations of the input and other tricks.
    - Another group of researchers put prior knowledge of invariances by putting in synthetic data. They put a lot of work into creating a huge amount of carefully designed extra training data (very instructive synthetic data).
        - so, for each training image, they transformed it to make more training examples by applying different transformations.
        - then, they trained a large net with many units per layer, many layers on a graphic processing unit (GPU), which gave them a 13 times fast in computation. Because of all synthetic data they put in, the net is not much overfit.
        - they combine three tricks successfully to form a large, deep and dumb net (connectivity, weight constraints and much more synthetic data) without much overfitting.
        - Putting the more effort on creating the synthetic data and train a large net on GPU, they managed to achieve 35 errors (with 5 errors of these 35 errors have 2 best bottom results are not the same as the actual digit)
![digit_error](images/digit_error.png)
        - With some more work on building several different models like this and then using a consensus to decide what the digit was, they managed to get down to about 25 errors that is around about the human error rate.

### 5.3.8. How to detect a significant drop in the error rate
- The question is how you can know if a model makes 30 errors in 10,000 test cases really significantly better than a model that makes 40 errors?
    - the answer is depends on which errors they make (the particular errors).
    - the number is just information, what you need to do is knowing which ones they get right and which ones they get wrong.
    - this statistical test is called as McNemar test that uses the particular errors and is much more powerful than a test that just uses the numbers of errors.
- Example: if 30 and 40 errors this could break down into [29 shared plus 1 vs. 11] or [15 shared plus 25 vs. 15]
![example_test](images/example_test.png)
For the above image, ignoring the black number, just consider the magnema number, let form a ratio of:
$$\frac{(n0)model-1-wrong,model-2-right}{(n0)model-1-right,model-2-wrong}=\frac{11}{1}$$
$\rightarrow$ model 2 is definitely better than model 1.
![example_test_2](images/example_test_2.png)
$$\frac{(n0)model-1-wrong,model-2-right}{(n0)model-1-right,model-2-wrong}=\frac{25}{15}=\frac{5}{3}$$
$\rightarrow$ the difference is not very significant so cannot be confident to tell that model 2 is better than model 1.

## 5.4. Convolutional neural networks for object recognition
<a id="conv_net_obj_reg"> </a>
From handwritten digits to 3D-objects recognition.
- Are kinds of nets developed to recognize handwritten digits could be scaled up to recognize objects in high resolution color images when the scene is cluttered or not?
    - The answer is you have to do things like **segmentation**, you have to deal with 3D viewpoint.

### 5.4.1. From hand-written digits to 3-D objects
- Recognizing real objects in color photographs is much more complicated than recognizing hand-written digits, because:
    - there are many more different kinds of objects, so there are maybe hundred times as many classes (1000 **objects** vs. 10 **digits**).
    - there are many more pixels, even if use sampled images, it also hundred times as many pixels (256x256 **color** vs. 28x28 **gray**).
    - two dimensional image of three-dimensional scene (what you got is a two-dimensional image of a three dimensional real scene, so a lot of information is being lost).
    - cluttered scenes such as overlapping (also in handwritten images), occlusion of large parts of objects by opaque other objects (not in handwritten images).
    - multiple objects in each image.
    - the condition of lighting variance.
- Will the same type of convolutional neural network work?

### 5.4.2. A neural network for ImageNet
- The dataset has 1.2 million high-resolution training images with 1,000 categories.
- Goal: the classification task to correctly label those images.
    - Because the image has different objects, so just get correctly class of top 5 guesses of 1,000 classes. This means that it's set to get right if one of those guesses corresponds to the label that a person has given the image (make 5 guesses about label).
![contest](images/contest.png)
- Goal: the localization task: (is not appeared in many computer vision systems that just can tell us what the object is, but cannot tell where this object is) for each guess, you have to place a box around the object once you've recognized it and the ratio of a box is considered to be right must be at least 50% overlap with the correct box.
![img_exp](images/img_exp.png)
- Some of the best existing computer vision methods were tried on this dataset by leading computer vision groups and soon discovered this task was very hard.
    - the computer vision systems typically use complicated multi-stage systems.
    - the early stages of these systems are typically hand-tuned by optimizing a few parameters using some of the data. The top stage of these systems is always a learning algorithm, but it doesn't learn all the way through in the same way that a deep neural net does when it's trained to do back propagation.
    - they don't have end-to-end learning, where the parameters used in the early feature detectors are being influenced by how useful they are for making final decision about classes. (no feedback line, no backpropagation).
![example_img](images/example_img.png)
Here are some examples from the test set. The predictors are un-normalized probabilities of deep-neural-network of [Alex Krizhevsky](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf)
        - The first and the second image, this net confident to classify kinds of class, but the last image (objects is isolated, no background), it is not confident to classify.
        - This network had error rate ~16% at classification task, ~34,1% at classification and localization tasks, better than more computer vision groups's solutions.

### Note: The difference between the traditional computer vision systems and deep learning network
1. Traditional Recognition Approach
![traditional_cv](images/traditional_cv.png)
This is a shallow architecture:
    - features are not learned, mostly based on predefined hand design.
    - trainable classifier is often generic (e.g. SVM).
2. Deep learning: learning the features
![deep_arch](images/deep_arch.png)
This is a deep architecture:
    - learn a **feature hierarchy** all the way from pixels to classifier.
    - each layer extracts features from the output of previous layer.
    - train all layers jointly.

### 5.4.3. The architecture of Alex's deep neural net (AlexNet)
- It is a very deep convolutional neural net with the architecture from the inspiration of the deep net used for digit recognition, then use for recognizing real objects:
    - deep: 7 hidden layers, not counting some max pooling layers.
    - the early layers were convolutional. (we could probably get away with using just local receptive fields without typing any weights if we have a much bigger computer. But by making them convolutionary, we cut down the parameters a lot, so we cut down the amount of training data a lot, which cuts down the amount of computation time a lot.)
    - the last two layers were globally connected and are where most of the parameters are. (about 60,000,000 parameters between these two layers).
        - these two layers are looking for combinations of the local feature extracted by the earlier layers.
    - the activation functions were:
        - **rectified linear units**(ReLUs) in every hidden layer, which train much faster and more expressive than logistic units (no logistic units anymore in deep learning network).
        - **competitive normalization** within a layer to suppress the hidden activities, if other nearby units have stronger activities. This helps a lot in variance of intensity. (in particular, for edge detector, you may got somewhat active due to some fairly faint edges (canh kha mo nhat) and that's pretty much irrelevant if having much more intense things around).

### 5.4.4. Tricks that significantly improve generalization
1. First, use the trick of enhancement of input data by transformations. Here is using down-sampling the images:
    - from (256x256) images, choose randomly (224x224) patches to get more data to train on (helping to deal with translation and variance), combined with left-right reflections of these images, which again doubled the amount of data.
        - not use dime reflection (phan chieu theo kieu dong ho, vong tron) because the gravity is very important.
        - the left-right reflection doesn't really change what things look like much unless the images are things like writing.
    - at test time, use ten different patches, not just one: the four (224x224) corner patches, the central (middle) which are five, plus the left-right reflections of these five patches, that gives ten patches. Then, run these 10 patches through the network and combine their opinions.
        - combine the opinions of 10 different (224x224) patches: 4 corners + center + 5 reflections.
    - at the top layer, where almost parameters are, use the new regularization technique (**dropout**), which is very effective and stops the network overfitting.
        - basic idea of dropout: each time presenting a training example, half of the hidden units in a layer are randomly removed. This means that the survivors (the other half of hidden units in this layer) cannot rely on too much on other hidden units, units cannot learn to overly correct for each other (cannot be too co-linear).
            - because if the cooperation of these hidden units are very strong, this is very good for fitting the training data. But if the test data distribution is very significantly different, then all that cooperations cause the overfitting. (prevents overfitting)
            - dropout stops hiddent units from replying on too much on other hidden units.